In [0]:
display(dbutils.fs.ls("dbfs:/user"))



path,name,size,modificationTime
dbfs:/user/ankit/,ankit/,0,0
dbfs:/user/hive/,hive/,0,0


In [0]:
# Choose a new folder (or delete old one)
new_path = "dbfs:/user/ankit/retail_db"
dbutils.fs.rm(new_path, True)  # Remove if exists

# Folder is now guaranteed empty
dbutils.fs.mkdirs(new_path)     # Optional: can let Delta create it automatically


True

In [0]:
%sql
-- 1️⃣ Drop the database if it exists (CASCADE removes all tables inside)
DROP DATABASE IF EXISTS itversity_retail_db CASCADE;

-- 2️⃣ Create the database in a separate folder
CREATE DATABASE itversity_retail_db
LOCATION 'dbfs:/user/ankit/retail_db';


In [0]:
%sql
-- Double-check no dangling path
DESCRIBE DATABASE EXTENDED itversity_retail_db;


database_description_item,database_description_value
Catalog Name,hive_metastore
Namespace Name,itversity_retail_db
Comment,
Location,dbfs:/user/ankit/retail_db
Owner,root
Properties,


In [0]:
%sql

USE itversity_retail_db;

DROP TABLE IF EXISTS orders;


In [0]:
%sql
-- 4️⃣ Create Delta table in managed warehouse
CREATE TABLE itversity_retail_db.orders (
  order_id BIGINT,
  order_date STRING,
  order_customer_id BIGINT,
  order_status STRING
)
USING delta;


com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: java.io.FileNotFoundException: /user/ankit/retail_db/orders/_delta_log/00000000000000000000.json
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:126)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:74)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.getFileStatus(DatabricksFileSystemV1.scala:281)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:216)
	at com.databricks.common.filesystem.LokiFileSystem.getFileStatus(LokiFileSystem.scala:314)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl$FileSystemBasedCommitCoordinatorClient.doCommit(OptimisticTransaction.scala:2621)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImplEdge$FileSystemBasedCommitCoordinatorClientEdge.doCommit(OptimisticTransactionImplEdge.scala:737)
	at com.dat

In [0]:
%fs ls user/ankit/retail_db/orders/_delta_log/_commits/

In [0]:
%sql
INSERT INTO itversity_retail_db.orders VALUES
(1, '2025-09-22', 101, 'DELIVERED');


org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `itversity_retail_db`.`orders` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 1 pos 12;
'InsertIntoStatement 'UnresolvedRelation [itversity_retail_db, orders], [], false, false, false, false
+- LocalRelation [col1#1433, col2#1434, col3#1435, col4#1436]

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.tableNotFound(package.scala:90)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$1(CheckAnalysis.scala:204)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$1$adapted(CheckAnalysis.scala:202)
	at org.apache.spark.sql.catalyst.trees.TreeNode